In [114]:
### Se importan las librerias
import os 
import pandas as pd
import numpy as np
from numpy import exp
import sys
sys.path.append('libreriasTesis')
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from statsmodels.tsa.api import VAR
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
from IPython.display import clear_output, Image, display
### Se asingan los paths
pathTesis = os.getcwd()
pathBases = os.path.join(pathTesis,'bases/ndatos')
pathGraficas = os.path.join(pathTesis,'GraficasN')

In [115]:
### Se lee la data y se cambian los nombres
dataTotalC = pd.read_csv(os.path.join(pathBases,'2014data.csv'), sep='|',index_col=['Fecha'], parse_dates=True)

In [116]:
dataTotalC=dataTotalC.drop(('0.5'), axis=1)
dataTotalC=dataTotalC.drop(('1.5'), axis=1)
dataTotalC=dataTotalC.drop(('2.5'), axis=1)
print("dimension:{}".format(dataTotalC.shape))

dimension:(365, 14)


In [117]:
##Drop nans column
dataTotal=dataTotalC.dropna(axis=1,how='all')
dataTotal=dataTotal.dropna(axis=0,how='any')

In [118]:
display(dataTotal.head())
print("dimension:{}".format(dataTotal.shape))

,0.004,0.1,0.3,1.8,1.9,2.5.1,4.8,4.9,5.7,6.6,8.3,10.6,12.7,14.3
Fecha,,,,,,,,,,,,,,
2014-01-15,3.145,3.156,3.164,4.54,4.82,4.97,6.16,5.86,6.22,6.72,6.74,6.88,7.47,7.05
2014-01-16,3.140,3.157,3.171,4.52,4.80,4.95,6.14,5.87,6.24,6.72,6.74,6.89,7.49,7.10
2014-01-17,3.139,3.155,3.173,4.53,4.80,4.98,6.17,5.89,6.25,6.79,6.80,6.97,7.57,7.19
2014-01-18,3.139,3.155,3.173,4.53,4.80,4.98,6.16,5.89,6.25,6.79,6.80,6.97,7.57,7.19
2014-01-19,3.139,3.155,3.173,4.52,4.80,4.98,6.16,5.89,6.25,6.78,6.80,6.97,7.57,7.19


dimension:(351, 14)


In [119]:
dataTotal.rename(columns={'2.5.1':'2.5'}, inplace=True)
print("dimension:{}".format(dataTotal.shape))

dimension:(351, 14)


In [120]:
dataTotal.tail()

,0.004,0.1,0.3,1.8,1.9,2.5,4.8,4.9,5.7,6.6,8.3,10.6,12.7,14.3
Fecha,,,,,,,,,,,,,,
2014-12-27,4.343,4.349,4.354,4.98,4.94,5.13,6.10,5.83,5.99,6.37,6.88,7.28,7.49,7.55
2014-12-28,4.343,4.349,4.354,4.98,4.94,5.13,6.10,5.83,5.99,6.37,6.88,7.28,7.49,7.55
2014-12-29,4.351,4.349,4.355,4.98,4.94,5.14,6.09,5.87,6.00,6.41,6.92,7.25,7.46,7.49
2014-12-30,4.358,4.352,4.360,4.98,4.93,5.14,6.10,5.90,5.99,6.45,6.92,7.26,7.47,7.49
2014-12-31,4.358,4.352,4.360,4.98,4.93,5.14,6.10,5.90,5.99,6.45,6.92,7.26,7.47,7.49


In [121]:
### Se Asignan los diccionarios corresponientes a la data
vencimientos=list(map(lambda x: float(x),list(dataTotal.columns.values)))
observadosY = dict(zip(dataTotal.columns, vencimientos))
### Segmentos
corto=3
mediano=10
cortoPlazoM=vencimientos[:corto]
medianoPlazoM=vencimientos[corto:mediano]
largoPlazoM=vencimientos[mediano:]
cortoPlazoY = dict(zip(dataTotal.columns[:corto], cortoPlazoM))
medianoPlazoY = dict(zip(dataTotal.columns[corto:mediano], medianoPlazoM))
largoPlazoY = dict(zip(dataTotal.columns[mediano:], largoPlazoM))
innerY = {'1.6':1.6, '8':8}

In [122]:
largoPlazoM

[8.3, 10.6, 12.7, 14.3]

In [123]:
# Estimation
lm = LinearRegression()
lm_no_intercept=LinearRegression(fit_intercept=False) #include or remove the intercept

# Se empieza a calculcar por separado los factores corresponientes a los modelos clasicos, segmentado y fuertemente segmentado

# ----------------------------------------------

### Nelson Siegel Clasico

In [124]:
### Los factores del modelo clasico con un lambda de 0.0609
def g(term,lambDa = 0.0609):
    resultado = (1 - exp(-lambDa*term)) / (lambDa*term)
    return resultado
def h(term,lambDa = 0.0609):
    resultado = ((1 - exp(-lambDa*term)) / (lambDa*term)) - exp(-lambDa*term)
    return resultado

In [125]:
def NS_clasico(data,terms):
    ### Se calcula el vector para los distinitos vencimientos
    gNS = g(np.array(terms))
    hNS = h(np.array(terms)) 
    X = pd.DataFrame({'x2':gNS,'x3':hNS}).values
    NS_fit=lm.fit(X,data.values.transpose())
    coefs=np.concatenate((np.array(NS_fit.intercept_.tolist()),NS_fit.coef_[:,0],NS_fit.coef_[:,1]),axis=0).reshape((3,data.values.shape[0])).transpose()
    betas_NS = pd.DataFrame(coefs, index=data.index,columns=["a","b","c"])
    ### Yields Estimados In-sample
    fitted_yield_NS = pd.DataFrame(NS_fit.predict(X).transpose(), index=data.index,columns=data.columns)    
    #RMSE
    NS3 = (pd.DataFrame(np.sqrt((data.values - fitted_yield_NS.values)**2)).apply(np.mean,0)*100).values
    RMSE_NS3=pd.Series(NS3,index=data.columns)
    out=(betas_NS,fitted_yield_NS,RMSE_NS3)
    return out

In [126]:
(betas_NS,fitted_yield_NS,RMSE_NS3)=NS_clasico(dataTotal,vencimientos)

In [ ]:
betas_NS.head()

In [ ]:
fitted_yield_NS.head()

In [ ]:
#Observed Yields
dataTotal.head()

# ----------------------------------------------

In [127]:
#Segmented Models
factores_seg=["a^I","b^I","c^I","a^II","b^II","c^II","a^III","b^III","c^III"]

### Nelson Siegel Segmented Non-Smootheness, prefered habitat model Vayanos and Vila (2009)

In [128]:
def NS_seg_Phabit(data,terms,corto,mediano,factores_seg):
    ### Se calcula el vector para los distinitos vencimientos
    gNS = g(np.array(terms))
    hNS = h(np.array(terms)) 
    X = pd.DataFrame({'x2':gNS,'x3':hNS}).values
    NS_fit_corto=lm.fit(X[:corto,:],data.iloc[:,:corto].values.transpose())
    NS_fit_mediano=lm.fit(X[corto:mediano,:],data.iloc[:,corto:mediano].values.transpose())
    NS_fit_largo=lm.fit(X[mediano:,:],data.iloc[:,mediano:].values.transpose())
    coefs_corto=np.concatenate((np.array(NS_fit_corto.intercept_.tolist()),NS_fit_corto.coef_[:,0],NS_fit_corto.coef_[:,1]),axis=0).reshape((3,data.values.shape[0])).transpose()
    coefs_mediano=np.concatenate((np.array(NS_fit_mediano.intercept_.tolist()),NS_fit_mediano.coef_[:,0],NS_fit_mediano.coef_[:,1]),axis=0).reshape((3,data.values.shape[0])).transpose()
    coefs_largo=np.concatenate((np.array(NS_fit_largo.intercept_.tolist()),NS_fit_largo.coef_[:,0],NS_fit_largo.coef_[:,1]),axis=0).reshape((3,data.values.shape[0])).transpose()
    betas_NS_seg_nosmooth = pd.DataFrame(np.concatenate((coefs_corto,coefs_mediano,coefs_largo),axis=1), index=data.index,columns=factores_seg)    
    ### Yields Estimados In-sample
    fited_yield_seg_nosmooth=np.concatenate((NS_fit_corto.predict(X[:corto,:]).transpose(),NS_fit_mediano.predict(X[corto:mediano,:]).transpose(),NS_fit_largo.predict(X[mediano:,:]).transpose()),axis=1)
    fitted_yield_NS_seg_nosmooth = pd.DataFrame(fited_yield_seg_nosmooth, index=data.index,columns=data.columns)
    #RMSE
    NS3_seg_nosmooth = (pd.DataFrame(np.sqrt((data.values - fitted_yield_NS_seg_nosmooth.values)**2)).apply(np.mean,0)*100).values
    RMSE_NS3_seg_nosmooth=pd.Series(NS3_seg_nosmooth,index=data.columns)
    out=(betas_NS_seg_nosmooth,fitted_yield_NS_seg_nosmooth,RMSE_NS3_seg_nosmooth)
    return out    

In [129]:
(betas_NS_seg_nosmooth,fitted_yield_NS_seg_nosmooth,RMSE_NS3_seg_nosmooth)=NS_seg_Phabit(dataTotal,vencimientos,corto,mediano,factores_seg)

In [ ]:
betas_NS_seg_nosmooth.head()

In [ ]:
fitted_yield_NS_seg_nosmooth.head()

In [ ]:
#Observed Yields
dataTotal.head()

In [130]:
print("Number of observed terms:{}".format(fitted_yield_NS_seg_nosmooth.columns.size))
print("Number of factors:{}".format(betas_NS_seg_nosmooth.columns.size))

Number of observed terms:14
Number of factors:9


# ----------------------------------------------

## Knots for Segments

In [131]:
#Introduce unobservable inner knots
Uobknots=('1.6','8')
cortoPlazoY.update({'1.6':1.6})
medianoPlazoY.update(innerY)
largoPlazoY.update({'8':8})
#introduce external knots
innerY.update( {'0.004':0.004,'15.3':15.3} )
#generalize
#dataTotal.columns[0],dataTotal.columns[(dataTotal.columns.size-1)]
#vencimientos[0],vencimientos[(len(vencimientos)-1)]

In [132]:
vec0 = lambda x: list(np.repeat(0,x))

### Nelson Siegel Segmented, impose smoothness by restrictions on the estimated coefficients

In [133]:
### Se crean las funciones de los factores y sus derivadas de nelson y siegel para el modelo Segmentado suave
def g(term,lambDa = 0.0609):
    resultado = (1 - exp(-lambDa*term)) / (lambDa*term)
    return resultado

def gPD(term,lambDa = 0.0609):
    resultado = (exp(-lambDa*term) * (lambDa*term + 1)) / (lambDa*(term ** 2))
    return resultado

def gSD(term,lambDa = 0.0609):
    resultado = (2 - exp(-lambDa*term) * (((lambDa * term) ** 2) + 2*(lambDa*term + 1))) / (lambDa * (term ** 3))
    return resultado 

def h(term,lambDa = 0.0609):
    resultado = ((1 - exp(-lambDa*term)) / (lambDa*term)) - exp(-lambDa*term)
    return resultado

def hPD(term,lambDa = 0.0609):
    resultado = (exp(-lambDa*term) * (((lambDa*term) ** 2) + (lambDa*term) + 1) - 1) / (lambDa*(term ** 2))
    return resultado

def hSD(term,lambDa = 0.0609):
    resultado = (2 - exp(-lambDa*term)*(((lambDa*term) ** 3) + ((lambDa*term) ** 2) + 2*(lambDa*term + 1))) / (lambDa * (term ** 3))
    return resultado


### Se crea la funcion X que permite retornar los vector de longitud 3 para la creacion de las matrices necesarias 
### del modelo segmentado


def X_function(tao,corte,signo,derivada):
    if derivada == '':
        if signo == '+':
            if corte == 'Corto':
                return [1, g(cortoPlazoY[tao]),h(cortoPlazoY[tao])]
            elif corte == 'Mediano':
                return [1, g(medianoPlazoY[tao]),h(medianoPlazoY[tao])]
            elif corte == 'Largo':
                return [1, g(largoPlazoY[tao]),h(largoPlazoY[tao])]
        elif signo == '-':
            if corte == 'Corto':
                return [-1, -g(cortoPlazoY[tao]),-h(cortoPlazoY[tao])]
            elif corte == 'Mediano':
                return [-1, -g(medianoPlazoY[tao]),-h(medianoPlazoY[tao])]
            elif corte == 'Largo':
                return [-1, -g(largoPlazoY[tao]),-h(largoPlazoY[tao])]
    elif derivada == 'PD':
        if signo == '+':
            if corte == 'Corto':
                return [0, gPD(cortoPlazoY[tao]),hPD(cortoPlazoY[tao])]
            elif corte == 'Mediano':
                return [0, gPD(medianoPlazoY[tao]),hPD(medianoPlazoY[tao])]
            elif corte == 'Largo':
                return [0, gPD(largoPlazoY[tao]),hPD(largoPlazoY[tao])]
        elif signo == '-':
            if corte == 'Corto':
                return [0, -gPD(cortoPlazoY[tao]),-hPD(cortoPlazoY[tao])]
            elif corte == 'Mediano':
                return [0, -gPD(medianoPlazoY[tao]),-hPD(medianoPlazoY[tao])]
            elif corte == 'Largo':
                return [0, -gPD(largoPlazoY[tao]),-hPD(largoPlazoY[tao])]
    elif derivada == 'SD':
        if signo == '+':
            if corte == 'Corto':
                return [0, gSD(cortoPlazoY[tao]),hSD(cortoPlazoY[tao])]
            elif corte == 'Mediano':
                return [0, gSD(medianoPlazoY[tao]),hSD(medianoPlazoY[tao])]
            elif corte == 'Largo':
                return [0, gSD(largoPlazoY[tao]),hSD(largoPlazoY[tao])]
        elif signo == '-':
            if corte == 'Corto':
                return [0, -gSD(cortoPlazoY[tao]),-hSD(cortoPlazoY[tao])]
            elif corte == 'Mediano':
                return [0, -gSD(medianoPlazoY[tao]),-hSD(medianoPlazoY[tao])]
            elif corte == 'Largo':
                return [0, -gSD(largoPlazoY[tao]),-hSD(largoPlazoY[tao])]
            

#### Aplicacion empirica

De aqui en adelante se crean las matrices necesarias w2, w1, R1, R2 para la creacion de Z.
Lo unico que cambia son los vencimientos de w1 y w2 y los valores de los yields

In [134]:
#R1
#complete matrix 
#NOTE the first line in matrix can be modified refering to refear to the first observable yield 
#this implies also a change in R2
R1_c =  np.matrix([X_function('1.6','Corto','+','')+ X_function('1.6','Mediano','-',''),
                   vec0(3)+X_function('8','Mediano','+',''),
                   X_function('1.6','Corto','+','PD')+ X_function('1.6','Mediano','-','PD'),
                   vec0(3)+X_function('8','Mediano','+','PD'),
                   X_function('1.6','Corto','+','SD')+ X_function('1.6','Mediano','-','SD')])
#truncate to make symetric
R1=R1_c[:,0:(R1_c.shape[1]-1)]
print("dimension:{}".format(R1.shape))

dimension:(5, 5)


In [135]:
#R2
#complete matrix  
#NOTE that the last line in matrix is the only one refering to an last observable yield (important to change for different year)
R2_c =  np.matrix([X_function('1.6','Mediano','-','')+vec0(3),
                 X_function('8','Mediano','+','')+ X_function('8','Largo','-',''),
                 X_function('1.6','Mediano','-','PD')+ vec0(3),
                 X_function('8','Mediano','+','PD')+ X_function('8','Largo','-','PD'),
                 X_function('1.6','Mediano','-','SD')+vec0(3)])
#truncate to make symetric
R2=R2_c[:,2:]
print("dimension:{}".format(R2.shape))

dimension:(5, 4)


In [136]:
#W matrix complete
W =  np.matrix([X_function('0.004','Corto','+','')+vec0(6),
                X_function('0.1','Corto','+','')+vec0(6),
                X_function('0.3','Corto','+','')+vec0(6),
                vec0(3) + X_function('1.8','Mediano','+','')+vec0(3),
                vec0(3) + X_function('1.9','Mediano','+','')+vec0(3),
                vec0(3) + X_function('2.5','Mediano','+','')+vec0(3),
                vec0(3) + X_function('4.8','Mediano','+','')+vec0(3),
                vec0(3) + X_function('4.9','Mediano','+','')+vec0(3),
                vec0(3) + X_function('5.7','Mediano','+','')+vec0(3),
                vec0(3) + X_function('6.6','Mediano','+','')+vec0(3),
                vec0(6) + X_function('8.3','Largo','+',''),
                vec0(6) + X_function('10.6','Largo','+',''),
                vec0(6) + X_function('12.7','Largo','+',''),
                vec0(6) + X_function('14.3','Largo','+','')])
print("dimension W:{}".format(W.shape))
W2=W[:,0:5]
print("dimension W2:{}".format(W2.shape))
W1=W[:,5:W.shape[1]]
print("dimension W1:{}".format(W1.shape))

dimension W:(14, 9)
dimension W2:(14, 5)
dimension W1:(14, 4)


In [137]:
#W matrix complete for estimating the unobserved yields
Wk =  np.matrix([X_function('1.6','Corto','+','')+vec0(6),
                vec0(3) + X_function('8','Mediano','+','')+vec0(3)])
#Wk =  np.matrix([vec0(3) + X_function('1.6','Mediano','+','')+vec0(3),
#                vec0(6) + X_function('8','Largo','+','')])
print("dimension W knots:{}".format(Wk.shape))
W2k=Wk[:,0:5]
print("dimension W2k:{}".format(W2k.shape))
W1k=Wk[:,5:W.shape[1]]
print("dimension W1k:{}".format(W1k.shape))

dimension W knots:(2, 9)
dimension W2k:(2, 5)
dimension W1k:(2, 4)


In [138]:
def NS_seg_weak(data,corto,mediano,R1,R2,W1,W2,W1k,W2k,factores_seg,Uobknots):
    R1_invertida = np.linalg.inv(R1)
    Z = W1 - (W2 * R1_invertida * R2)
    #print("dimension Z:{}".format(Z.shape))
    #estimation of the restricted factors for "c^II","a^III","b^III","c^III"
    NS_Seg_M_L=lm_no_intercept.fit(Z,data.values.transpose())
    #use equation from the paper to obtain factors #"a^I","b^I","c^I","a^II","b^II"
    coefs_S_M=(-R1_invertida*R2*NS_Seg_M_L.coef_.transpose()).transpose()
    betas_NS_seg_smooth = pd.DataFrame(np.concatenate((coefs_S_M,NS_Seg_M_L.coef_),axis=1), index=data.index,columns=factores_seg)
    ### Yields Estimados In-sample
    fitted_yield_NS_seg_smooth_z = pd.DataFrame(NS_Seg_M_L.predict(Z).transpose(), index=data.index,columns=data.columns)
    #RMSE
    NS3_seg_smooth = (pd.DataFrame(np.sqrt((data.values - fitted_yield_NS_seg_smooth_z.values)**2)).apply(np.mean,0)*100).values
    RMSE_NS3_seg_smooth=pd.Series(NS3_seg_smooth,index=data.columns)
    #Latent Yields
    Zk = W1k - (W2k * R1_invertida * R2)
    #print("dimension Z knots:{}".format(Zk.shape))
    ZZ_invertida = np.linalg.inv(Z.transpose()*Z)
    fit_yield_latent=(Zk*ZZ_invertida*Z.transpose()*data.values.transpose()).transpose()
    fitted_yield_latent = pd.DataFrame(fit_yield_latent, index=data.index,columns=Uobknots)
    #Add Latent Yields
    short=fitted_yield_NS_seg_smooth_z.iloc[:,:corto]
    knotSM=fitted_yield_latent.iloc[:,0]
    medium=fitted_yield_NS_seg_smooth_z.iloc[:,corto:mediano]
    knotML=fitted_yield_latent.iloc[:,1]
    long=fitted_yield_NS_seg_smooth_z.iloc[:,mediano:]
    frames = [short, knotSM, medium,knotML,long]
    fitted_yields_NS_seg_weak= pd.concat(frames,axis=1)
    out=(betas_NS_seg_smooth,fitted_yields_NS_seg_weak,RMSE_NS3_seg_smooth)
    return out       

In [139]:
(betas_NS_seg_smooth,fitted_yield_NS_seg_weak,RMSE_NS3_seg_smooth)=NS_seg_weak(dataTotal,corto,mediano,R1,R2,W1,W2,W1k,W2k,factores_seg,Uobknots)

In [ ]:
betas_NS_seg_smooth.head()

In [ ]:
#Observed Yields
dataTotal.head()

In [ ]:
fitted_yield_NS_seg_weak.head()

# ----------------------------------------------

## Nelson Siegel Strongly Segmented

In [140]:
### Se crean las funciones de los factores y sus derivadas de nelson y siegel para el modelo Segmentado Fuertemente
### Segmentado

def gs(term,termImenos,lambDa = 0.0609, p = 0.5):
    A = term - termImenos*(1-p)
    resultado = (1 - exp(-lambDa*A)) / (lambDa*A)
    return resultado

def gPDs(term,termImenos,lambDa = 0.0609, p = 0.5):
    A = term - termImenos*(1-p)
    resultado = (exp(-lambDa*A) * (lambDa*A + 1)) / (lambDa*(A ** 2))
    return resultado

def gSDs(term,termImenos,lambDa = 0.0609, p = 0.5):
    A = term - termImenos*(1-p)
    resultado = (2 - exp(-lambDa*A) * (((lambDa * A) ** 2) + 2*(lambDa*A + 1))) / (lambDa * (A ** 3))
    return resultado 

def hs(term,termImenos,lambDa = 0.0609, p = 0.5):
    A = term - termImenos*(1-p)
    resultado = ((1 - exp(-lambDa*A)) / (lambDa*A)) - exp(-lambDa*A)
    return resultado

def hPDs(term,termImenos,lambDa = 0.0609, p = 0.5):
    A = term - termImenos*(1-p)
    resultado = (exp(-lambDa*A) * (((lambDa*A) ** 2) + (lambDa*A) + 1) - 1) / (lambDa*(A ** 2))
    return resultado

def hSDs(term,termImenos,lambDa = 0.0609, p = 0.5):
    A = term - termImenos*(1-p)
    resultado = (2 - exp(-lambDa*A)*(((lambDa*A) ** 3) + ((lambDa*A) ** 2) + 2*(lambDa*A + 1))) / (lambDa * (A ** 3))
    return resultado


### Se crea la funcion X que permite retornar los vector de longitud 3 para la creacion de las matrices necesarias 
### del modelo fuertemente segmentado

def Xs_function(tao,taoImenos,corte,signo,derivada):
    if derivada == '':
        if signo == '+':
            if corte == 'Corto':
                return [1, gs(cortoPlazoY[tao],cortoPlazoY[taoImenos]),hs(cortoPlazoY[tao],cortoPlazoY[taoImenos])]
            elif corte == 'Mediano':
                return [1, gs(medianoPlazoY[tao],medianoPlazoY[taoImenos]),hs(medianoPlazoY[tao],medianoPlazoY[taoImenos])]
            elif corte == 'Largo':
                return [1, gs(largoPlazoY[tao],largoPlazoY[taoImenos]),hs(largoPlazoY[tao],largoPlazoY[taoImenos])]
        elif signo == '-':
            if corte == 'Corto':
                return [-1, -gs(cortoPlazoY[tao],cortoPlazoY[taoImenos]),-hs(cortoPlazoY[tao],cortoPlazoY[taoImenos])]
            elif corte == 'Mediano':
                return [-1, -gs(medianoPlazoY[tao],medianoPlazoY[taoImenos]),-hs(medianoPlazoY[tao],medianoPlazoY[taoImenos])]
            elif corte == 'Largo':
                return [-1, -gs(largoPlazoY[tao],largoPlazoY[taoImenos]),-hs(largoPlazoY[tao],largoPlazoY[taoImenos])]
    elif derivada == 'PD':
        if signo == '+':
            if corte == 'Corto':
                return [0, gPDs(cortoPlazoY[tao],cortoPlazoY[taoImenos]),hPDs(cortoPlazoY[tao],cortoPlazoY[taoImenos])]
            elif corte == 'Mediano':
                return [0, gPDs(medianoPlazoY[tao],medianoPlazoY[taoImenos]),hPDs(medianoPlazoY[tao],medianoPlazoY[taoImenos])]
            elif corte == 'Largo':
                return [0, gPDs(largoPlazoY[tao],largoPlazoY[taoImenos]),hPDs(largoPlazoY[tao],largoPlazoY[taoImenos])]
        elif signo == '-':
            if corte == 'Corto':
                return [0, -gPDs(cortoPlazoY[tao],cortoPlazoY[taoImenos]),-hPDs(cortoPlazoY[tao],cortoPlazoY[taoImenos])]
            elif corte == 'Mediano':
                return [0, -gPDs(medianoPlazoY[tao],medianoPlazoY[taoImenos]),-hPDs(medianoPlazoY[tao],medianoPlazoY[taoImenos])]
            elif corte == 'Largo':
                return [0, -gPDs(largoPlazoY[tao],largoPlazoY[taoImenos]),-hPDs(largoPlazoY[tao],largoPlazoY[taoImenos])]
    elif derivada == 'SD':
        if signo == '+':
            if corte == 'Corto':
                return [0, gSDs(cortoPlazoY[tao],cortoPlazoY[taoImenos]),hSDs(cortoPlazoY[tao],cortoPlazoY[taoImenos])]
            elif corte == 'Mediano':
                return [0, gSDs(medianoPlazoY[tao],medianoPlazoY[taoImenos]),hSDs(medianoPlazoY[tao],medianoPlazoY[taoImenos])]
            elif corte == 'Largo':
                return [0, gSDs(largoPlazoY[tao],largoPlazoY[taoImenos]),hSDs(largoPlazoY[tao],largoPlazoY[taoImenos])]
        elif signo == '-':
            if corte == 'Corto':
                return [0, -gSDs(cortoPlazoY[tao],cortoPlazoY[taoImenos]),-hSDs(cortoPlazoY[tao],cortoPlazoY[taoImenos])]
            elif corte == 'Mediano':
                return [0, -gSDs(medianoPlazoY[tao],medianoPlazoY[taoImenos]),-hSDs(medianoPlazoY[tao],medianoPlazoY[taoImenos])]
            elif corte == 'Largo':
                return [0, -gSDs(largoPlazoY[tao],largoPlazoY[taoImenos]),-hSDs(largoPlazoY[tao],largoPlazoY[taoImenos])]

#### Aplicacion empirica

De aqui en adelante se crean las matrices necesarias w2, w1, R1, R2 para la creacion de Z.
Lo unico que cambia son los vencimientos de w1 y w2 y los valores de los yields

In [141]:
#R1
#complete matrix 
#NOTE the first line in matrix can be modified refering to refear to the first observable yield 
#this implies also a change in R2
R1_c =  np.matrix([Xs_function('1.6','0.004','Corto','+','')+ Xs_function('1.6','1.6','Mediano','-',''),
                   vec0(3)+Xs_function('8','1.6','Mediano','+',''),
                   Xs_function('1.6','0.004','Corto','+','PD')+ Xs_function('1.6','1.6','Mediano','-','PD'),
                   vec0(3)+Xs_function('8','1.6','Mediano','+','PD'),
                   Xs_function('1.6','0.004','Corto','+','SD')+ Xs_function('1.6','1.6','Mediano','-','SD')])
#truncate to make symetric
R1=R1_c[:,0:(R1_c.shape[1]-1)]
R1_invertida = np.linalg.inv(R1)
print("dimension:{}".format(R1.shape))

dimension:(5, 5)


In [142]:
#R2
#complete matrix  
#NOTE that the last line in matrix is the only one refering to an last observable yield (important to change for different year)
R2_c =  np.matrix([Xs_function('1.6','1.6','Mediano','-','')+vec0(3),
                 Xs_function('8','1.6','Mediano','+','')+ Xs_function('8','8','Largo','-',''),
                 Xs_function('1.6','1.6','Mediano','-','PD')+ vec0(3),
                 Xs_function('8','1.6','Mediano','+','PD')+ Xs_function('8','8','Largo','-','PD'),
                 Xs_function('1.6','1.6','Mediano','-','SD')+vec0(3)])
#truncate to make symetric
R2=R2_c[:,2:]
print("dimension:{}".format(R2.shape))

dimension:(5, 4)


In [143]:
#W matrix complete
W =  np.matrix([X_function('0.004','Corto','+','')+vec0(6),
                X_function('0.1','Corto','+','')+vec0(6),
                X_function('0.3','Corto','+','')+vec0(6),
                vec0(3) + X_function('1.8','Mediano','+','')+vec0(3),
                vec0(3) + X_function('1.9','Mediano','+','')+vec0(3),
                vec0(3) + X_function('2.5','Mediano','+','')+vec0(3),
                vec0(3) + X_function('4.8','Mediano','+','')+vec0(3),
                vec0(3) + X_function('4.9','Mediano','+','')+vec0(3),
                vec0(3) + X_function('5.7','Mediano','+','')+vec0(3),
                vec0(3) + X_function('6.6','Mediano','+','')+vec0(3),
                vec0(6) + X_function('8.3','Largo','+',''),
                vec0(6) + X_function('10.6','Largo','+',''),
                vec0(6) + X_function('12.7','Largo','+',''),
                vec0(6) + X_function('14.3','Largo','+','')])
print("dimension W:{}".format(W.shape))
W2=W[:,0:5]
print("dimension W2:{}".format(W2.shape))
W1=W[:,5:W.shape[1]]
print("dimension W1:{}".format(W1.shape))

dimension W:(14, 9)
dimension W2:(14, 5)
dimension W1:(14, 4)


In [144]:
#W matrix complete for estimating the unobserved yields
Wk =  np.matrix([Xs_function('1.6','0.004','Corto','+','')+vec0(6),
                vec0(3) + Xs_function('8','1.6','Mediano','+','')+vec0(3)])
#Wk =  np.matrix([vec0(3) + Xs_function('1.6','1.6','Mediano','+','')+vec0(3),
#                vec0(6) + Xs_function('8','8','Largo','+','')])
print("dimension W knots:{}".format(Wk.shape))
W2k=Wk[:,0:5]
print("dimension W2k:{}".format(W2k.shape))
W1k=Wk[:,5:W.shape[1]]
print("dimension W1k:{}".format(W1k.shape))

dimension W knots:(2, 9)
dimension W2k:(2, 5)
dimension W1k:(2, 4)


In [145]:
def NS_seg_strong(data,corto,mediano,R1,R2,W1,W2,W1k,W2k,factores_seg,Uobknots):
    R1_invertida = np.linalg.inv(R1)
    Z = W1 - (W2 * R1_invertida * R2)
    #estimation of the restricted factors for "c^II","a^III","b^III","c^III"
    NS_Seg_M_L=lm_no_intercept.fit(Z,data.values.transpose())
    #use equation from the paper to obtain factors #"a^I","b^I","c^I","a^II","b^II"
    coefs_S_M=(-R1_invertida*R2*NS_Seg_M_L.coef_.transpose()).transpose()
    betas_NS_seg_smooth_s = pd.DataFrame(np.concatenate((coefs_S_M,NS_Seg_M_L.coef_),axis=1), index=data.index,columns=factores_seg)
    ### Yields Estimados In-sample
    fitted_yield_NS_seg_smooth_s_z = pd.DataFrame(NS_Seg_M_L.predict(Z).transpose(), index=data.index,columns=data.columns)
    #RMSE
    NS3_seg_smooth_s = (pd.DataFrame(np.sqrt((data.values - fitted_yield_NS_seg_smooth_s_z.values)**2)).apply(np.mean,0)*100).values
    RMSE_NS3_seg_smooth_s=pd.Series(NS3_seg_smooth_s,index=data.columns)
    Zk = W1k - (W2k * R1_invertida * R2)
    #print("dimension Z knots:{}".format(Zk.shape))
    #Fitted latent yields
    ZZ_invertida = np.linalg.inv(Z.transpose()*Z)
    fit_yield_latent_s=(Zk*ZZ_invertida*Z.transpose()*data.values.transpose()).transpose()
    fitted_yield_latent_s = pd.DataFrame(fit_yield_latent_s, index=data.index,columns=Uobknots)
    #Add Latent Yields
    short=fitted_yield_NS_seg_smooth_s_z.iloc[:,:corto]
    knotSM=fitted_yield_latent_s.iloc[:,0]
    medium=fitted_yield_NS_seg_smooth_s_z.iloc[:,corto:mediano]
    knotML=fitted_yield_latent_s.iloc[:,1]
    long=fitted_yield_NS_seg_smooth_s_z.iloc[:,mediano:]
    frames = [short, knotSM, medium,knotML,long]
    fitted_yields_NS_seg_strong= pd.concat(frames,axis=1)
    out=(betas_NS_seg_smooth_s,fitted_yields_NS_seg_strong,RMSE_NS3_seg_smooth_s)
    return out 

In [146]:
(betas_NS_seg_smooth_s,fitted_yield_NS_seg_strong,RMSE_NS3_seg_smooth_s)=NS_seg_strong(dataTotal,corto,mediano,R1,R2,W1,W2,W1k,W2k,factores_seg,Uobknots)

In [ ]:
betas_NS_seg_smooth_s.head()

In [ ]:
#Observed Yields
dataTotal.head()

In [ ]:
fitted_yield_NS_seg_strong.head()

# ----------------------------------------------

# Graficas

### Loadings Segmentado

In [ ]:
def g(term,lambDa = 0.0609):
    resultado = (1 - exp(-lambDa*term)) / (lambDa*term)
    return resultado


def h(term,lambDa = 0.0609):
    resultado = ((1 - exp(-lambDa*term)) / (lambDa*term)) - exp(-lambDa*term)
    return resultado

maturityPaper = np.array(range(0,121))
gValor = g(maturityPaper)
hValor = h(maturityPaper)
vec1 = list(np.repeat(1,len(maturityPaper)))
grafica2 = pd.DataFrame.from_dict({'1':vec1,'g':gValor,'h':hValor})
grafica2.plot()
plt.xlabel('Maturity (Months)')
plt.savefig(os.path.join(pathGraficas,'FactorLoadingsNS3'))
plt.show()

### Loadings Segmentado Fuertemente

In [ ]:
def g(term,termImenos,lambDa = 0.0609, p = 0.5):
    A = term - termImenos*(1-p)
    resultado = (1 - exp(-lambDa*A)) / (lambDa*A)
    return resultado

def h(term,termImenos,lambDa = 0.0609, p = 0.5):
    A = term - termImenos*(1-p)
    resultado = ((1 - exp(-lambDa*A)) / (lambDa*A)) - exp(-lambDa*A)
    return resultado

maturityPaper1 = np.array(range(0,15))
maturityPaper2 = np.array(range(16,54))
maturityPaper3 = np.array(range(55,107))
maturityPaper4 = np.array(range(108,120))

gValor1 = g(maturityPaper1,0)
gValor2 = g(maturityPaper2,16)
gValor3 = g(maturityPaper3,55)
gValor4 = g(maturityPaper4,108)
gValor = np.append(np.append(gValor1,gValor2),np.append(gValor3,gValor4))

hValor1 = h(maturityPaper1,0)
hValor2 = h(maturityPaper2,16)
hValor3 = h(maturityPaper3,55)
hValor4 = h(maturityPaper4,108)
hValor = np.append(np.append(hValor1,hValor2),np.append(hValor3,hValor4))

vec1 = list(np.repeat(1,len(hValor)))

grafica2 = pd.DataFrame.from_dict({'1':vec1,'g':gValor,'h':hValor})
grafica2.plot()
plt.axvline(x=16)
plt.axvline(x=55)
plt.axvline(x=108)
plt.xlabel('Maturity (Months)')
plt.savefig(os.path.join(pathGraficas,'FactorLoadingsNS3Strong'))
plt.show()

### Observed Yields

In [ ]:
dataTotal.index.names = ['Date']
plt.ylabel('Yield')
ax = plt.subplot(111)
for col in dataTotal.columns:
    ax.plot(dataTotal.index,dataTotal.loc[:,col].values,label=col)
ax.legend(loc='upper left', bbox_to_anchor=(1, 1), shadow=True, ncol=1)
plt.xticks(rotation='vertical')
plt.savefig(os.path.join(pathGraficas,'Observed yields'))
plt.show()

### Modelo Segmentado (Fitted Yield)

In [ ]:
#fitSegYields=fitted_yield_NS # Nelson & Siegel Clasico
#fitSegYields=fitted_yield_NS_seg_nosmooth # Prefered Habitat
#fitSegYields=fitted_yield_NS_seg_weak # Weakly Segmented
fitSegYields=fitted_yield_NS_seg_strong #Strongly Segmented

In [ ]:
fitSegYields.index.names = ['Date']
plt.ylabel('Yield')
ax = plt.subplot(111)
for col in fitSegYields.columns:
    ax.plot(fitSegYields.index,fitSegYields.loc[:,col].values,label=col)
ax.legend(loc='upper left', bbox_to_anchor=(1, 1), shadow=True, ncol=1)
plt.xticks(rotation='vertical')
plt.savefig(os.path.join(pathGraficas,'Fitted yields'))
plt.show()

## RMSE entre NS3 y NS3 Segmented Fitted Yields Insample

In [147]:
RMSE_tabl=pd.concat([RMSE_NS3,RMSE_NS3_seg_nosmooth,RMSE_NS3_seg_smooth,RMSE_NS3_seg_smooth_s],axis=1)
RMSE_tabl.columns= ['NS3', 'NS3_S', 'NS3_W_S','NS3_S_S'] 
RMSE_tabl.index.names = ['Maturity']

In [148]:
RMSE_tabl

,NS3,NS3_S,NS3_W_S,NS3_S_S
Maturity,,,,
0.004,12.810582,78.811373,6.334184,5.704994
0.1,9.559727,75.082000,3.483315,2.336625
0.3,10.535877,70.562873,8.365602,6.639436
1.8,14.573967,59.068183,8.426317,8.290449
1.9,19.895397,58.658727,12.725753,3.565818
2.5,14.734361,49.959150,5.193382,3.643037
4.8,17.294800,39.054814,13.615293,21.796980
4.9,20.138903,29.515699,23.564023,15.368019
5.7,10.673819,25.903231,10.342213,5.501363


In [149]:
#MSE_Comparativo.to_latex(index=False,bold_rows=True ,buf=os.path.join(pathGraficas,'tablaInSample'))

In [150]:
#Export betas
betas_NS.to_csv(os.path.join(pathBases,'NS_2014.csv'), sep='|')
betas_NS_seg_smooth_s.to_csv(os.path.join(pathBases,'NS_S_2014.csv'), sep='|')

## Out-of-Sample Forecasting (Dynamic 1-month, 21 days)

In [151]:
from statsmodels.tsa.arima_model import ARIMA

In [152]:
dataTotal.shape

(351, 14)

In [153]:
def NS_betas_fore(data,beta):
    betasout=pd.DataFrame(index=data.index)
    for col in beta.columns:
        model = ARIMA(beta.loc[:,col].values, order=(1,0,0))
        model_fit = model.fit(disp=0)
        betasout.loc[:,col]=model_fit.forecast(steps=dataTotal.shape[0])[0]
    return betasout  

In [154]:
### Se lee la data y se cambian los nombres
betas_NS_Y_1 = pd.read_csv(os.path.join(pathBases,'NS_2013.csv'), sep='|',index_col=['Date'], parse_dates=True)
betas_NS_seg_Y_1 = pd.read_csv(os.path.join(pathBases,'NS_S_2013.csv'), sep='|',index_col=['Date'], parse_dates=True)

In [155]:
betasout=NS_betas_fore(dataTotal,betas_NS_Y_1)

In [ ]:
betasout.head()

In [156]:
#Factor Loadings NS Clasic
gNS = g(np.array(vencimientos))
hNS = h(np.array(vencimientos)) 
X = pd.DataFrame({'x1':np.ones_like(gNS),'x2':gNS,'x3':hNS}).values
### Yields Estimados Out-sample
fitted_yield_NS_out = pd.DataFrame(np.dot(X,betasout.values.transpose()).transpose(), index=dataTotal.index,columns=dataTotal.columns)
#RMSE
NS3_out = (pd.DataFrame(np.sqrt((dataTotal.values - fitted_yield_NS_out.values)**2)).apply(np.mean,0)*100).values
RMSE_NS3_out=pd.Series(NS3_out,index=dataTotal.columns)    

In [ ]:
fitted_yield_NS_out.head()

In [158]:
betasout_s=NS_betas_fore(dataTotal,betas_NS_seg_Y_1)

In [157]:
#Factor Loadings NS Segmented
Z = W1 - (W2 * R1_invertida * R2)
### Yields Estimados Out-sample
fitted_yield_NS_seg_out = pd.DataFrame(np.dot(Z,betasout_s.iloc[:,5:].values.transpose()).transpose(), index=dataTotal.index,columns=dataTotal.columns)
#RMSE
NS3_seg_out = (pd.DataFrame(np.sqrt((dataTotal.values - fitted_yield_NS_seg_out.values)**2)).apply(np.mean,0)*100).values
RMSE_NS3_seg_out=pd.Series(NS3_seg_out,index=dataTotal.columns)    

In [159]:
#RMSE Out-of-Sample Dynamic
RMSE_tabl_out=pd.concat([RMSE_NS3_out,RMSE_NS3_seg_out],axis=1)
RMSE_tabl_out.columns= ['NS3', 'NS3_S_S'] 
RMSE_tabl_out.index.names = ['Maturity']

In [160]:
RMSE_tabl_out

,NS3,NS3_S_S
Maturity,,
0.004,55.331209,86.919483
0.1,57.031455,87.316872
0.3,58.181214,82.414518
1.8,61.118869,44.521651
1.9,67.001301,51.434648
2.5,56.713110,47.745063
4.8,50.673500,53.880827
4.9,23.220865,25.300384
5.7,25.022243,27.848419


## Out-of-Sample Forecasting (Rolling)

In [219]:
#Join Betas of year_t-1 and t
betas_NS.shape

(351, 3)

In [220]:
betas_NS_Y_1.shape

(329, 3)

In [221]:
betas_NS.shape

(351, 3)

In [222]:
betas_NS_Y_1.tail()

,a,b,c
Date,,,
2013-12-27,-19.880488,22.942354,47.196042
2013-12-28,-19.859895,22.923269,47.164524
2013-12-29,-19.775645,22.837692,47.057939
2013-12-30,-20.135767,23.172328,47.891620
2013-12-31,-20.080815,23.118465,47.816876


In [223]:
betas_t_1=betas_NS_Y_1

In [224]:
betas_t=betas_NS

In [225]:
betas_FS=betas_t_1.append(betas_t)

In [226]:
betas_FS.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 680 entries, 2013-02-06 to 2014-12-31
Data columns (total 3 columns):
a    680 non-null float64
b    680 non-null float64
c    680 non-null float64
dtypes: float64(3)
memory usage: 21.2 KB


In [227]:
lastObs_t_1=betas_t_1.shape[0]-1

In [228]:
lastObs_t_1

328

In [229]:
betas_FS.iloc[lastObs_t_1,:]

a   -20.080815
b    23.118465
c    47.816876
Name: 2013-12-31 00:00:00, dtype: float64

In [230]:
WE=126 # estimation window
WT=betas_t.shape[0]  #evaluation window

In [231]:
WT

351

In [232]:
#Forecast Sample
betas_FS=betas_FS.iloc[((lastObs_t_1)-(WE-5)):,:]

In [233]:
betas_FS.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 473 entries, 2013-09-01 to 2014-12-31
Data columns (total 3 columns):
a    473 non-null float64
b    473 non-null float64
c    473 non-null float64
dtypes: float64(3)
memory usage: 14.8 KB


In [247]:
#N&S Classic
#for i in range(0,dataTotal.shape[0]):
#    data=betas_FS.iloc[(0+i):(WE+i),:]

fsteps=5
i=5
data=betas_FS.iloc[(0+i):(WE+i),:]
betasout=pd.DataFrame(index=dataTotal.index[i:i+fsteps],columns=data.columns)
ObsYields=dataTotal.iloc[i:i+fsteps,:]
for col in data.columns:
    model = ARIMA(data.loc[:,col].values, order=(1,0,0))
    model_fit = model.fit(disp=0)
    betasout.loc[:,col]=model_fit.forecast(steps=fsteps)[0]
#fitted_yield = pd.DataFrame(np.dot(Z,betasout.iloc[:,5:].values.transpose()).transpose(), index=ObsYields.index,columns=dataTotal.columns)
fitted_yield = pd.DataFrame(np.dot(X,betasout.values.transpose()).transpose(), index=ObsYields.index,columns=dataTotal.columns)
Err_yield=pd.DataFrame(ObsYields.values - fitted_yield.values, index=ObsYields.index,columns=dataTotal.columns)    


In [248]:
Err_yield

Maturity,0.004,0.1,0.3,1.8,1.9,2.5,4.8,4.9,5.7,6.6,8.3,10.6,12.7,14.3
Fecha,,,,,,,,,,,,,,
2014-01-20,-0.011523,-0.062165,-0.207919,0.060345,0.276031,0.078984,0.138813,-0.181126,-0.085321,0.174965,-0.238359,-0.314882,0.221019,-0.038773
2014-01-21,-0.007461,-0.053089,-0.197824,0.111096,0.316730,0.119303,0.146655,-0.163424,-0.088836,0.209902,-0.206805,-0.248709,0.301643,0.017307
2014-01-22,0.004389,-0.044211,-0.186900,0.110918,0.326518,0.118824,0.154202,-0.155994,-0.092424,0.135000,-0.234620,-0.281217,0.234248,-0.024111
2014-01-23,0.015157,-0.032405,-0.179024,0.129893,0.345475,0.147609,0.191444,-0.148848,-0.066125,0.160189,-0.191929,-0.242616,0.138550,-0.003369
2014-01-24,0.034963,-0.022553,-0.170085,0.198095,0.383672,0.195715,0.178375,-0.101995,0.010030,0.215413,-0.108843,-0.193085,0.284303,0.049236


In [177]:
betasout

,a^I,b^I,c^I,a^II,b^II,c^II,a^III,b^III,c^III
Fecha,,,,,,,,,
2014-01-15,11.862672,-8.798661,14.534696,5.890174,-2.201639,16.975583,7.002077,-0.424574,2.582441
2014-01-16,12.137903,-9.073751,14.347834,5.963407,-2.270466,16.914377,7.026788,-0.444402,2.510915
2014-01-17,12.399342,-9.335105,14.173832,6.033019,-2.335856,16.858527,7.050787,-0.463159,2.442021
2014-01-18,12.647681,-9.583410,14.011804,6.099189,-2.397981,16.807562,7.074095,-0.480902,2.375663
2014-01-19,12.883575,-9.819316,13.860927,6.162088,-2.457004,16.761056,7.096731,-0.497686,2.311748


In [ ]:
ObsYields

In [ ]:
fitted_yield

In [ ]:
Err_yield

In [ ]:
Err_yield.iloc[(fsteps-1),:]

In [179]:
betas_FS.head()

,a^I,b^I,c^I,a^II,b^II,c^II,a^III,b^III,c^III
2013-08-28,42.171130,-39.121726,-5.572115,13.981207,-9.788405,8.953987,10.175965,-2.670893,-2.654297
2013-08-29,49.344814,-46.307430,-11.001116,15.874431,-11.586246,6.428497,10.757808,-3.226572,-3.295839
2013-08-30,42.708001,-39.667267,-5.964311,14.117649,-9.924898,8.781159,10.202018,-2.716300,-2.541045
2013-08-31,42.647429,-39.607772,-5.909206,14.101034,-9.910013,8.811681,10.198013,-2.711462,-2.539406
2013-09-01,42.647429,-39.607772,-5.909206,14.101034,-9.910013,8.811681,10.198013,-2.711462,-2.539406


In [249]:
ErrEW = pd.DataFrame(columns=dataTotal.columns)
fsteps=5
NS_model=0
#N&S Classic
for i in range(0,(dataTotal.shape[0]-fsteps)):
    data=betas_FS.iloc[(0+i):(WE+i),:]
    betasout=pd.DataFrame(index=dataTotal.index[i:i+fsteps],columns=data.columns)
    ObsYields=dataTotal.iloc[i:i+fsteps,:]
    for col in data.columns:
        model = ARIMA(data.loc[:,col].values, order=(1,0,0))
        model_fit = model.fit(disp=0)
        betasout.loc[:,col]=model_fit.forecast(steps=fsteps)[0]
    if NS_model==0:
        fitted_yield = pd.DataFrame(np.dot(X,betasout.values.transpose()).transpose(), index=ObsYields.index,columns=dataTotal.columns)
    else:
        fitted_yield = pd.DataFrame(np.dot(Z,betasout.iloc[:,5:].values.transpose()).transpose(), index=ObsYields.index,columns=dataTotal.columns)
    Err_yield=pd.DataFrame(ObsYields.values - fitted_yield.values, index=ObsYields.index,columns=dataTotal.columns)
    ErrEW=ErrEW.append(Err_yield.iloc[(fsteps-1),:])

C:\ccastro\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:649: RuntimeWarning: divide by zero encountered in true_divide
  R_mat, T_mat)
C:\ccastro\lib\site-packages\statsmodels\tsa\tsatools.py:584: RuntimeWarning: overflow encountered in exp
  newparams = ((1-np.exp(-params))/
C:\ccastro\lib\site-packages\statsmodels\tsa\tsatools.py:585: RuntimeWarning: overflow encountered in exp
  (1+np.exp(-params))).copy()
C:\ccastro\lib\site-packages\statsmodels\tsa\tsatools.py:585: RuntimeWarning: invalid value encountered in true_divide
  (1+np.exp(-params))).copy()
C:\ccastro\lib\site-packages\statsmodels\tsa\tsatools.py:586: RuntimeWarning: overflow encountered in exp
  tmp = ((1-np.exp(-params))/
C:\ccastro\lib\site-packages\statsmodels\tsa\tsatools.py:587: RuntimeWarning: overflow encountered in exp
  (1+np.exp(-params))).copy()
C:\ccastro\lib\site-packages\statsmodels\tsa\tsatools.py:587: RuntimeWarning: invalid value encountered in true_divide
  (1+np.exp(-params))).copy()
C:\c

In [193]:
ErrEW.tail()

Maturity,0.004,0.1,0.3,1.8,1.9,2.5,4.8,4.9,5.7,6.6,8.3,10.6,12.7,14.3
2014-12-26,0.098217,0.062963,-0.016853,0.019743,-0.056742,-0.078656,0.182529,-0.114823,-0.164019,0.000211,0.154721,0.169962,0.111702,0.013008
2014-12-27,0.100823,0.065244,-0.015246,0.016456,-0.060346,-0.084132,0.170265,-0.127369,-0.178781,-0.016966,0.133203,0.142991,0.080143,-0.021818
2014-12-28,0.098814,0.061569,-0.022300,-0.012228,-0.090250,-0.120808,0.115652,-0.182497,-0.237301,-0.077839,0.071716,0.087565,0.035981,-0.054378
2014-12-29,0.099715,0.055901,-0.024079,0.002818,-0.074245,-0.089614,0.138478,-0.109437,-0.193118,-0.003888,0.141322,0.074304,0.005202,-0.131516
2014-12-30,0.105940,0.058256,-0.019460,0.004066,-0.082909,-0.087801,0.151378,-0.076515,-0.200080,0.039145,0.143995,0.085860,0.015219,-0.132944


In [ ]:
dataTotal.shape

In [ ]:
def NS_yields_fore(Yields,betasS,horizon,WE,X,Z,NS_model):
    ErrEW = []
    for i in range(0,(Yields.shape[0]-horizon)):
        data=betasS.iloc[(0+i):(WE+i),:]
        betasout=pd.DataFrame(index=Yields.index[i:i+horizon],columns=data.columns)
        ObsYields=Yields.iloc[i:i+horizon,:]
        for col in data.columns:
            model = ARIMA(data.loc[:,col].values, order=(1,0,0))
            model_fit = model.fit(disp=0)
            betasout.loc[:,col]=model_fit.forecast(steps=horizon)[0]
        if NS_model==0:
            fitted_yield = pd.DataFrame(np.dot(X,betasout.values.transpose()).transpose(), index=ObsYields.index,columns=Yields.columns)
        else:
            fitted_yield = pd.DataFrame(np.dot(Z,betasout.iloc[:,5:].values.transpose()).transpose(), index=ObsYields.index,columns=Yields.columns)
        Err_yield=pd.DataFrame(ObsYields.values - fitted_yield.values, index=ObsYields.index,columns=Yields.columns)
        ErrEW.append(Err_yield)
    return ErrEW

In [ ]:
ErrEW_5=NS_yields_fore(dataTotal,betas_FS,5,WE,X,Z,0)

In [ ]:
len(ErrEW_5)

In [ ]:
ErrEW_5